In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

sys.path.append('D:/Dropbox/py_projects/customlib/')
import customlib as cl

import numpy as np
import scipy.constants as pc

import matplotlib.pyplot as plt
%matplotlib widget
import ipywidgets as widgets

from matplotlib.widgets import Slider, Button

In [2]:
def func_trip_symm(maglen, qgrad1, qgrad2, qgrad3, driftlen, invecx, invecy):

    ENG0 = np.sqrt(200**2 + cl.mc2**2)

    MAG_LEN = maglen * np.array([1, 2, 1])
    # MAG_GRAD = 70
    MAG_GRAD = np.array([qgrad1, qgrad2, qgrad3])
    MAG_POS = np.array([  0.0
                        , MAG_LEN[0]/2 + driftlen + MAG_LEN[1]/2
                        , 2*(MAG_LEN[0]/2 + driftlen + MAG_LEN[1]/2)
                        ])

    ELEMENT_LIST = []

    qprops = {   'eletype':'quad'
                ,'position':MAG_POS[0]
                ,'maglen':MAG_LEN[0]
                ,'gradient':MAG_GRAD[0]
                ,'totengMeV':ENG0
                ,'restmassMeV':cl.mc2
                ,'charge':1
                ,'focus':'focus'
                    }
    ele_q1 = cl.BLElement('Q1', eleprops=qprops)
    ELEMENT_LIST.append(ele_q1)

    qprops = qprops.copy() # need copy, otherwise element props change
    qprops['position'] = MAG_POS[1]
    qprops['maglen'] = MAG_LEN[1]
    qprops['gradient'] = MAG_GRAD[1]
    qprops['focus'] = 'defocus'

    ele_q2 = cl.BLElement('Q2', eleprops=qprops)
    ELEMENT_LIST.append(ele_q2)

    qprops = qprops.copy()
    qprops['position'] = MAG_POS[2]
    qprops['maglen'] = MAG_LEN[2]
    qprops['gradient'] = MAG_GRAD[2]
    qprops['focus'] = 'focus'
    ele_q3 = cl.BLElement('Q3', eleprops=qprops)
    ELEMENT_LIST.append(ele_q3)

    # make the beamline
    bl1 = cl.BeamLine()
    for ele in ELEMENT_LIST:
        bl1.add_element(ele)
    
    # output steps
    S_IN = -100e-3
    S_OUT = 1.2
    S_STEP = 100e-3

    outpos, outvecx = bl1.ray_trace(invecx, S_IN, np.arange(S_IN, S_OUT, S_STEP), ytransport=False)
    outpos, outvecy = bl1.ray_trace(invecy, S_IN, np.arange(S_IN, S_OUT, S_STEP), ytransport=True)
    
    return outpos, outvecx, outvecy

In [3]:
# set the RNG seed for consistency with each run
rng = np.random.default_rng(42)
NPART = 20
E_CENX = 0.0 # (m) init centroid of electron beam
E_SIGX = 95.0e-6 # (m) init size of electron beam
E_CENXP = 0.0 # (m) init centroid of electron beam in x-momentum 
E_SIGXP = 100.0e-6 # (m) init size of electron beam in x-momentum 
E_INVEC_X = np.transpose(np.c_[ rng.normal(E_CENX, E_SIGX, NPART), rng.normal(E_CENXP, E_SIGXP, NPART) ] )
NSIGMA = 3
E_INVEC_X = np.transpose(np.c_[ 
                                  np.linspace( E_CENX-NSIGMA*E_SIGX, E_CENX+NSIGMA*E_SIGX,NPART )
                                , np.linspace( E_CENXP-NSIGMA*E_SIGXP, E_CENXP+NSIGMA*E_SIGXP,NPART ) 
                                ] 
                                )
E_INVEC_X.shape

(2, 20)

In [78]:
outpos, outvecx, outvecy = func_trip_symm(40e-3, 80,70,80, 30e-3, E_INVEC_X, E_INVEC_X)


In [79]:
fig, ax = plt.subplots(figsize=(9,7))
colorlist = ['dodgerblue', 'orange', 'dodgerblue']
ls = dict(linewidth=1.50,linestyle='-',marker='o',ms=0, alpha=0.5)

yscale = 1e6

xvec = outpos
for i in range(NPART):
    ls['color'] = 'dodgerblue'
    yvec = yscale*outvecx[:,0,i]
    ax.plot(xvec, yvec, **ls)

    ls['color'] = 'orange'
    yvec = yscale*outvecy[:,0,i]
    ax.plot(xvec, yvec, **ls)

fs = dict(fontsize=18, color='k')
plt.xlabel('z (m)',**fs)
plt.ylabel('x,y (m)', **fs)

titlestr = ('triplet')
plt.title(titlestr, fontsize=24)



pr = np.array([-200, 200, -125, 125])
# plt.axis(pr)

# xticks = np.arange(-200,200,50)
# ax.set_xticks(xticks)
# yticks = np.arange(-120, 120, 20)
# ax.set_yticks(yticks)
ax.tick_params(labelcolor='k', labelsize=18, width=1)

plt.grid(False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
g1_init = 72
g2_init = 63
g3_init = 70
outpos, outvecx, outvecy = func_trip_symm(40e-3, g1_init,g2_init,g3_init, 30e-3, E_INVEC_X, E_INVEC_X)


fig, ax = plt.subplots(figsize=(9,7))
plt.subplots_adjust(left=0.15, bottom=0.5)
colorlist = ['dodgerblue', 'orange', 'dodgerblue']
ls = dict(linewidth=1.50,linestyle='-',marker='o',ms=0, alpha=0.5)

xvec = outpos
for i in range(NPART):
    ls['color'] = 'dodgerblue'
    yvec = yscale * outvecx[:,0,i]
    ax.plot(xvec, yvec, **ls)

    ls['color'] = 'orange'
    yvec = yscale * outvecy[:,0,i]
    ax.plot(xvec, yvec, **ls)

ax.legend(['x','y'], fontsize=13,loc=(0.0,0.6))
        
fs = dict(fontsize=18, color='k')
ax.set_xlabel('z (m)',**fs)
plt.ylabel('x,y (m)', **fs)

titlestr = ('triplet')
ax.set_title(titlestr, fontsize=24)

plt.margins(x=0)
pr = np.array([-100e-3, 1.1, -250, 250])
ax.axis(pr)

# xticks = np.arange(-200,200,50)
# ax.set_xticks(xticks)
# yticks = np.arange(-120, 120, 20)
# ax.set_yticks(yticks)
ax.tick_params(labelcolor='k', labelsize=18, width=1)

axcolor = 'lightgray'
axg1 = plt.axes([0.25, 0.2, 0.65, 0.03], facecolor=axcolor)
axg2 = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)
axg3 = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)

sliderg1 = Slider(axg1, 'Grad 1', 0.0, 100.0, valinit=g1_init, valstep=1, dragging=False)
sliderg2 = Slider(axg2, 'Grad 2', 0.0, 100.0, valinit=g2_init, valstep=1, dragging=False)
sliderg3 = Slider(axg3, 'Grad 3', 0.0, 100.0, valinit=g3_init, valstep=1, dragging=False)




yscale = 1e6

def update(val):
    """Remove old lines from plot and plot new one"""
    # [l.remove() for l in ax.lines]
    ax.clear()

    outpos, outvecx, outvecy = func_trip_symm(40e-3, sliderg1.val, sliderg2.val, sliderg3.val, 30e-3, E_INVEC_X, E_INVEC_X)
    
    xvec = outpos
    for i in range(NPART):
        ls['color'] = 'dodgerblue'
        yvec = yscale * outvecx[:,0,i]
        ax.plot(xvec, yvec, **ls)

        ls['color'] = 'orange'
        yvec = yscale * outvecy[:,0,i]
        ax.plot(xvec, yvec, **ls)

        ax.axis(pr)

sliderg1.on_changed(update)
sliderg2.on_changed(update)
sliderg3.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sliderg1.reset()
    sliderg2.reset()
    sliderg3.reset()
button.on_clicked(reset)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
71,63,71


200

In [68]:
len(ax.lines)

40

In [20]:
from matplotlib.widgets import Slider, Button

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
delta_f = 5.0
s = a0 * np.sin(2 * np.pi * f0 * t)
l, = plt.plot(t, s, lw=2)
ax.margins(x=0)

axcolor = 'lightgray'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0, valstep=delta_f, dragging=False)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)


def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    fig.canvas.draw_idle()


sfreq.on_changed(update)
samp.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …